In [ ]:
#Observers: Sasha Brownsberger, Robert Lupton, Chris Stubbs 

In [1]:
#Do the basic imports for general AuxTel operations
import asyncio
from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

#And the imports needed specifically for focusing
#import numpy as np 
import matplotlib.pyplot as plt 
import time 
import numpy as np 
import math 
import scipy.optimize as optimize
%matplotlib inline 

#And additional start up script
domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)
await asyncio.gather(attcs.start_task, latiss.start_task)

Could not read historical data in 60.19 sec
Could not read historical data in 60.97 sec
Could not read historical data in 61.89 sec
RemoteTelemetry(ATDome, 0, position) falling behind; read 10 messages
Could not read historical data in 63.94 sec


[[None, None, None, None, None, None, None], [None, None, None, None]]

In [ ]:
#And enable the system:
await attcs.enable()
await latiss.enable()

In [3]:
#Slew to target object
target_name = "HD 185975"
await attcs.slew_object(name = target_name,  rot_sky=0.0, slew_timeout=240.)

In [5]:
#Take an image to see where it is 
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='ronchi90lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


In [6]:
#Adjust xy to get the star at the right spot 
await attcs.offset_xy(x = 24, y = -53)

In [7]:
#Take an image to see where it is 
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='ronchi90lpmm')

In [8]:
#Focus units are given in mm 
focus_center = -0.2155
focus_step = 0.02
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * math.floor(n_steps / 2), focus_center + focus_step * math.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str(focus_positions))

focus_positions = [-0.2955 -0.2755 -0.2555 -0.2355 -0.2155 -0.1955 -0.1755 -0.1555 -0.1355]


In [12]:
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

In [13]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    focus_image = await latiss.take_engtest(exptime=2., n=1, filter='empty_1', grating='empty_1') 
    print ('Newest image id is = ' +str(focus_image) )
    focus_images[i] = int(focus_image)
    
#This resets the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955
Newest image id is = [2020021900133]
Working on focus position -0.27549999999999997


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


Newest image id is = [2020021900134]
Working on focus position -0.2555
Newest image id is = [2020021900135]
Working on focus position -0.2355
Newest image id is = [2020021900136]
Working on focus position -0.2155


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


Newest image id is = [2020021900137]
Working on focus position -0.1955
Newest image id is = [2020021900138]
Working on focus position -0.1755
Newest image id is = [2020021900139]
Working on focus position -0.1555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


Newest image id is = [2020021900140]
Working on focus position -0.1355
Newest image id is = [2020021900141]


In [14]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    focus_image = await latiss.take_engtest(exptime=5., n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(focus_image) )
    focus_images[i] = int(focus_image)
    
#This resets the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955
Newest image id is = [2020021900142]
Working on focus position -0.27549999999999997
Newest image id is = [2020021900143]
Working on focus position -0.2555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


Newest image id is = [2020021900144]
Working on focus position -0.2355
Newest image id is = [2020021900145]
Working on focus position -0.2155


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 76 messages


Newest image id is = [2020021900146]
Working on focus position -0.1955
Newest image id is = [2020021900147]
Working on focus position -0.1755


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


Newest image id is = [2020021900148]
Working on focus position -0.1555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


Newest image id is = [2020021900149]
Working on focus position -0.1355
Newest image id is = [2020021900150]


In [15]:
#adust x, y to get object at center for Tiago's pointing model 
await attcs.offset_xy(x = 26, y = 35)

In [16]:
#Let's verify we're there 
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='empty_1')

In [17]:
#adust x, y to get object at center for Tiago's pointing model 
await attcs.offset_xy(x = 0, y = 0)

In [18]:
#Let's verify we're there 
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='empty_1')
print ('Sequence number = ' + (science_images))

In [19]:
#adust x, y to get object at center for Tiago's pointing model 
await attcs.offset_xy(x = 34, y = -20)

In [20]:
#Let's verify we're there 
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='empty_1')
print ('Sequence number = ' + (science_images))

In [21]:
#Help Tiago
await attcs.atptg.cmd_pointNewFile.start() 

In [22]:
await attcs.atptg.cmd_pointAddData.start() 

In [23]:
#Adjust xy to get the star at the right spot 
await attcs.offset_xy(x = 24, y = -53)

In [25]:
#Let's verify we're there 
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


science_image = [2020021900155]


In [26]:
#Adjust xy to get the star at the right spot 
await attcs.offset_xy(x = 8, y = -40)

In [27]:
#Let's verify we're there 
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

science_image = [2020021900156]


In [ ]:
#Now we take measurements for each of the gratings and filters. 
#First, all filters
science_image = await latiss.take_object(exptime=2., n=3, filter='quadnotch1',grating='empty_1')
print ('science_image = ' + str(science_image))

In [28]:
#Focus units are given in mm 
#Let's try another focus run.  Current best guess: 0.468
focus_center = -0.468
focus_step = 0.02
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * math.floor(n_steps / 2), focus_center + focus_step * math.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str(focus_positions))

focus_positions = [-0.548 -0.528 -0.508 -0.488 -0.468 -0.448 -0.428 -0.408 -0.388]


In [29]:

#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    focus_image = await latiss.take_engtest(exptime=5., n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(focus_image) )
    focus_images[i] = int(focus_image)
    
#This resets the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.548


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 21 messages


Newest image id is = [2020021900157]
Working on focus position -0.528
Newest image id is = [2020021900158]
Working on focus position -0.508
Newest image id is = [2020021900159]
Working on focus position -0.48800000000000004
Newest image id is = [2020021900160]
Working on focus position -0.468
Newest image id is = [2020021900161]
Working on focus position -0.448


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


Newest image id is = [2020021900162]
Working on focus position -0.42800000000000005
Newest image id is = [2020021900163]
Working on focus position -0.40800000000000003
Newest image id is = [2020021900164]
Working on focus position -0.388
Newest image id is = [2020021900165]


In [ ]:
#Argh still bad.  I must have gone the wrong way. 
#First focus position: -0.2155 + 0.3
#Quoted best fit position: Where center appears in plot: ~0.3
#Quoted best fit focus in plot: 0.468 
#So best fit focus I should enter is 0.468 - 5.1 ~ -0.04


In [30]:
#Focus units are given in mm 
#Let's try another focus run.  Current best guess: 0.468
focus_center = -0.04
focus_step = 0.02
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * math.floor(n_steps / 2), focus_center + focus_step * math.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str(focus_positions))

focus_positions = [-0.12 -0.1  -0.08 -0.06 -0.04 -0.02  0.    0.02  0.04]


In [31]:

#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    focus_image = await latiss.take_engtest(exptime=5., n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(focus_image) )
    focus_images[i] = int(focus_image)
    
#This resets the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.12
Newest image id is = [2020021900166]
Working on focus position -0.09999999999999999
Newest image id is = [2020021900167]
Working on focus position -0.07999999999999999
Newest image id is = [2020021900168]
Working on focus position -0.06
Newest image id is = [2020021900169]
Working on focus position -0.039999999999999994
Newest image id is = [2020021900170]
Working on focus position -0.01999999999999999
Newest image id is = [2020021900171]
Working on focus position 0.0
Newest image id is = [2020021900172]
Working on focus position 0.020000000000000018
Newest image id is = [2020021900173]
Working on focus position 0.04
Newest image id is = [2020021900174]


In [32]:
#Now adjust to apparent best focus center at: 
# claimed best focus: 0.43 

await attcs.ataos.cmd_applyFocusOffset.set_start(offset = -0.07)

In [33]:
#Robert gives another image adjustment... 
#Adjust xy to get the star at the right spot 
await attcs.offset_xy(x = 19, y = 5)

In [34]:
#Now we take measurements for each of the gratings and filters. 
#First, all filters
science_image = await latiss.take_object(exptime=5., n=3, filter='quadnotch1',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 85 messages


science_image = [2020021900175 2020021900176 2020021900177]


In [35]:
#Now we take measurements for each of the gratings and filters. 
#First, all filters
science_image = await latiss.take_object(exptime=5., n=3, filter='BG40',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


science_image = [2020021900178 2020021900179 2020021900180]


In [36]:
#Now we take measurements for each of the gratings and filters. 
#First, all filters
science_image = await latiss.take_object(exptime=5., n=3, filter='RG610',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


science_image = [2020021900181 2020021900182 2020021900183]


In [37]:
#Those were in the wrong positions because I did not first go to 0.  
#Let's redo those when the star is in the proper place.
await attcs.offset_xy(x = 0, y = 0)

In [38]:
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


science_image = [2020021900184]


In [39]:
#Now adjust, since I first put it at x=0, y=0
await attcs.offset_xy(x = 10, y = -35)

In [40]:
science_image = await latiss.take_object(exptime=2., n=3, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


science_image = [2020021900185 2020021900186 2020021900187]


In [41]:
science_image = await latiss.take_object(exptime=5., n=3, filter='quadnotch1',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


science_image = [2020021900188 2020021900189 2020021900190]


In [42]:
science_image = await latiss.take_object(exptime=5., n=3, filter='BG40',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


science_image = [2020021900191 2020021900192 2020021900193]


In [43]:
science_image = await latiss.take_object(exptime=5., n=3, filter='RG610',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


science_image = [2020021900194 2020021900195 2020021900196]


In [44]:
#Let's reset the offset.  First, move to 0.0
await attcs.offset_xy(x = 0, y = 0)

In [45]:
#And take test exposure
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

science_image = [2020021900197]


In [46]:
#Let's reset the offset.  First, move to 0.0
await attcs.offset_xy(x = 9, y = -34)

In [47]:
#And take test exposure
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

science_image = [2020021900198]


In [48]:
#Let's reset the offset.  We want to do that WITH THE GRATING IN
await attcs.offset_xy(x = 0, y = 0)

In [49]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900199]


In [50]:
#That seems to be a long way off...
#Let's reset the offset.  We want to do that WITH THE GRATING IN
await attcs.offset_xy(x = 27, y = -32)

In [51]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900200]


In [52]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=3, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


science_image = [2020021900201 2020021900202 2020021900203]


In [53]:
#Let's reset the offset.  We want to do that WITH THE GRATING IN
await attcs.offset_xy(x = 0, y = 0)

In [54]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='ronchi170lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900204]


In [55]:
#Let's reset the offset.  We want to do that WITH THE GRATING IN
await attcs.offset_xy(x = 27, y = -33)

In [56]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='ronchi170lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900205]


In [57]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=3, filter='empty_1',grating='ronchi170lpmm')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


science_image = [2020021900206 2020021900207 2020021900208]


In [58]:
await attcs.stop_tracking() 

In [59]:
#Now need to shuffle pointing component.  Send to standby then back to enable 
await salobj.set_summary_state(attcs.atptg, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [61]:
#Now need to shuffle pointing component.  Send to standby then back to enable 
await salobj.set_summary_state(attcs.atptg, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [62]:
await attcs.atptg.cmd_focusName.set_start(focus=3) #focus = 3 selects Nasmyth port of pointing 

In [63]:
#Tiago added a more complex pointing model as an option 
#Slew to target object
target_name = "HD 185975"
await attcs.slew_object(name = target_name,  rot_sky=0.0, slew_timeout=240.)

In [64]:
#And take test exposure
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

science_image = [2020021900209]


RemoteEvent(ATMCS, 0, target) falling behind; read 17 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 11 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 24 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


In [65]:
#Tiago suggestion
target_name = "HD 87983"
await attcs.slew_object(name = target_name,  rot_sky=0.0, slew_timeout=240.)

RemoteEvent(ATMCS, 0, target) falling behind; read 16 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 17 messages


In [66]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


science_image = [2020021900216]


RemoteEvent(ATMCS, 0, target) falling behind; read 58 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 13 messages


In [67]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

science_image = [2020021900217]


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 18 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 18 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 17 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 86 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 31 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 30 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


In [ ]:
#Measured offset is x=-11, y = 9 to boresight. 

In [68]:
#Tiago suggestion
target_name = "HD 68601"
await attcs.slew_object(name = target_name,  rot_sky=0.0, slew_timeout=240.)

RemoteEvent(ATMCS, 0, target) falling behind; read 17 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 17 messages


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1563232222, ack=<SalRetCode.CMD_FAILED: -302>, error=6611, result='Rejected : rotator position angle out of range')

In [69]:
###IF you get the above error (rotator position angle out of range), change rot_sky from 0.0 to -180.0 or +180.0 (vice versa) 
#Tiago suggestion
target_name = "HD 68601"
await attcs.slew_object(name = target_name,  rot_sky=-180.0, slew_timeout=240.)

In [70]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

science_image = [2020021900226]


offset: 15, -1 

In [71]:
#Tiago suggestion
target_name = "HD 28464"
await attcs.slew_object(name = target_name,  rot_sky=-180.0, slew_timeout=240.)

RemoteEvent(ATMCS, 0, target) falling behind; read 17 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 17 messages


In [72]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

science_image = [2020021900227]


offset: -10, -2 

In [73]:
await attcs.stop_tracking()

In [74]:
#Going to load pointing model manually. 
await attcs.atptg.cmd_pointLoadModel.set_start(pointingFile="at_20200219_manual.mod")

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1275506718, ack=<SalRetCode.CMD_FAILED: -302>, error=6613, result='Rejected : unable to open input file')

In [75]:
await attcs.atptg.cmd_pointLoadModel.set_start(pointingFile="repos/ts_pointing_common/install/data/at_20200219_manual.mod")

In [76]:
#Tiago suggestion
target_name = "HD 28464"
await attcs.slew_object(name = target_name,  rot_sky=-180.0, slew_timeout=240.)

In [77]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


science_image = [2020021900228]


Robert offset from boresight: -4, -11

In [78]:
#Tiago suggestion
target_name = "HD 68601"
await attcs.slew_object(name = target_name,  rot_sky=-180.0, slew_timeout=240.)

In [79]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

science_image = [2020021900229]


Robert offset from boresight: 21, 3

In [80]:
#Tiago suggestion
target_name = "HD 87983"
await attcs.slew_object(name = target_name,  rot_sky=0.0, slew_timeout=240.)

TimeoutError: 

In [81]:
#Tiago suggestion
target_name = "HD 87983"
await attcs.slew_object(name = target_name,  rot_sky=0.0, slew_timeout=240.)

In [82]:
#And take test exposure
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

science_image = [2020021900230]


Robert offset from boresight: -2, -13

In [83]:
#Robert offset the image using his own automated version. 
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='empty_1')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


science_image = [2020021900231]


RemoteEvent(ATMCS, 0, target) falling behind; read 18 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 41 messages


Looks good! Nicely done Robert.  

In [84]:
#Slew to target object
target_name = "HD 185975"
await attcs.slew_object(name = target_name,  rot_sky=0.0, slew_timeout=240.)

In [85]:
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900232]


In [86]:
#Robert offset the image using his own automated version. 
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900233]


In [90]:
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


science_image = [2020021900234]


In [91]:
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

In [92]:
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900235]


In [93]:
#Robert moved it
science_image = await latiss.take_object(exptime=5., n=1, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900236]


In [96]:
#Focus units are given in mm 
focus_center = -0.0155
focus_step = 0.05
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * math.floor(n_steps / 2), focus_center + focus_step * math.ceil(n_steps / 2 - 1), n_steps)
print ('focus_positions = ' + str(focus_positions))

focus_positions = [-0.2155 -0.1655 -0.1155 -0.0655 -0.0155  0.0345  0.0845  0.1345  0.1845]


In [98]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    focus_image = await latiss.take_engtest(exptime=15., n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(focus_image) )
    focus_images[i] = int(focus_image)
    
#This resets the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.21550000000000002


TimeoutError: 

In [99]:
#This likely happened because we applied the offset that was already recorded. 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = 0.0)

In [100]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    focus_image = await latiss.take_engtest(exptime=15., n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(focus_image) )
    focus_images[i] = int(focus_image)
    
#This resets the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.21550000000000002


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


Newest image id is = [2020021900246]
Working on focus position -0.16550000000000004


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


Newest image id is = [2020021900247]
Working on focus position -0.11550000000000002


...

Newest image id is = [2020021900249]
Working on focus position -0.015500000000000014


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


Newest image id is = [2020021900250]
Working on focus position 0.034499999999999975
Newest image id is = [2020021900251]
Working on focus position 0.08450000000000002
Newest image id is = [2020021900252]
Working on focus position 0.1345
Newest image id is = [2020021900253]
Working on focus position 0.1845


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


Newest image id is = [2020021900254]


In [101]:
#Chris thinks 248: -0.11550000000000002 or 249: -0.0655
#We'll go with -0.1155 as our value. 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = -0.1155)

In [102]:
#Let's reset the offset.  We want to do that WITH THE GRATING IN
await attcs.offset_xy(x = -32, y = -15, persistent = True)

TypeError: offset_xy() got an unexpected keyword argument 'persistent'

In [ ]:
#I have not yet loaded the new offset_xy option.  This should work tomorrow when I reload the kernel. 

In [103]:
#Robert moved it
science_image = await latiss.take_object(exptime=15., n=1, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900255]


In [104]:
#Looks good
science_image = await latiss.take_object(exptime=15., n=1, filter='quadnotch1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900256]


In [105]:
#Looks good
science_image = await latiss.take_object(exptime=15., n=1, filter='BG40',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900257]


In [106]:
#Looks good
science_image = await latiss.take_object(exptime=120., n=1, filter='BG40',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900258]


In [107]:
#Looks good
science_image = await latiss.take_object(exptime=15., n=1, filter='RG610',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900259]


In [108]:
#Looks good
science_image = await latiss.take_object(exptime=120., n=1, filter='RG610',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


science_image = [2020021900260]


In [109]:
#Slew to object again, rotating system to see if display still looks good. 
target_name = "HD 185975"
await attcs.slew_object(name = target_name,  rot_sky=-180.0, slew_timeout=240.)

In [110]:
#And reexpose 
science_image = await latiss.take_object(exptime=15., n=1, filter='RG610',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


science_image = [2020021900261]


In [ ]:
#Rotation on Firefly display seems to work fine. 
await attcs.offset_azel(az = , el = , ) #units are arcseconds 

In [112]:
await attcs.offset_xy(x = 0, y = 0)

In [113]:
target_name = "HD 185975"
await attcs.slew_object(name = target_name,  rot_sky=-0.0, slew_timeout=240.)

In [ ]:
#Rotation on Firefly display seems to work fine. 
await attcs.offset_azel(az = , el = , ) #units are arcseconds 

In [115]:
#And reexpose 
science_image = await latiss.take_object(exptime=15., n=1, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


science_image = [2020021900262]


In [117]:
#Rotation on Firefly display seems to work fine. 
await attcs.offset_azel(az = 0, el = 30) #units are arcseconds 

In [118]:
#And reexpose 
science_image = await latiss.take_object(exptime=15., n=1, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


science_image = [2020021900263]


In [119]:
#Now go perform this test at the zenith 
target_name = "HD 68450"
await attcs.slew_object(name = target_name,  rot_sky=-0.0, slew_timeout=240.)

RemoteEvent(ATMCS, 0, target) falling behind; read 16 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 17 messages


In [120]:
#And reexpose 
science_image = await latiss.take_object(exptime=10., n=1, filter='empty_1',grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900264]


In [121]:
#Now go perform this test at the zenith 
target_name = "HD 125745"
await attcs.slew_object(name = target_name,  rot_pa=0.0, slew_timeout=240.)

RemoteEvent(ATMCS, 0, target) falling behind; read 11 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 13 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 10 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 41 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 12 messages


In [122]:
#And slew to object with rot_pa = 0.0  
science_image = await latiss.take_object(exptime=10., n=1, filter='empty_1', grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


science_image = [2020021900265]


In [123]:
#Rotation on Firefly display seems to work fine. 
await attcs.offset_azel(az = 0, el = 30) #units are arcseconds 

In [124]:
#And slew to object with rot_pa = 0.0  
science_image = await latiss.take_object(exptime=10., n=1, filter='empty_1', grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900266]


In [125]:
#Now go perform this test at the zenith 
target_name = "HD 125745"
await attcs.slew_object(name = target_name,  rot_pa=90.0, slew_timeout=240.)

In [126]:
#And slew to object with rot_pa = 0.0  
science_image = await latiss.take_object(exptime=10., n=1, filter='empty_1', grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900267]


In [127]:
#Rotation on Firefly display seems to work fine. 
await attcs.offset_azel(az = 0, el = 30) #units are arcseconds 

In [128]:
#And slew to object with rot_pa = 0.0  
science_image = await latiss.take_object(exptime=10., n=1, filter='empty_1', grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


science_image = [2020021900268]


In [129]:
#Now go pback to zenith with rot_pa=+90
target_name = "HD 68450"
await attcs.slew_object(name = target_name,  rot_pa=90.0, slew_timeout=240.)

In [130]:
#And slew to object with rot_pa = 0.0  
science_image = await latiss.take_object(exptime=10., n=1, filter='empty_1', grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900269]


In [131]:
#Rotation on Firefly display seems to work fine. 
await attcs.offset_azel(az = 0, el = 30) #units are arcseconds 

In [132]:
#And slew to object with rot_pa = 0.0  
science_image = await latiss.take_object(exptime=10., n=1, filter='empty_1', grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

science_image = [2020021900270]


In [133]:
#Now go pback to zenith with rot_pa=+90
target_name = "HD 109978"
await attcs.slew_object(name = target_name,  rot_pa=90.0, slew_timeout=240.)

CancelledError: 

In [134]:
#Dome seems to have stopped.  Let's see if we can move the dome
await attcs.stop_tracking() #First stop tracking

In [135]:
#Try to move dome directly 
await attcs.atdome.cmd_moveAzimuth.set_start(azimuth = 150.0) #Sent to where it thinks it was

In [136]:
#Now try to slew again to the target.  Tracking should automatically resume. 
target_name = "HD 109978"
await attcs.slew_object(name = target_name,  rot_pa=90.0, slew_timeout=240.)

In [ ]:
#And if the system stops tracking completely. 

In [137]:
#And slew to object with rot_pa = 0.0  
science_image = await latiss.take_object(exptime=10., n=1, filter='empty_1', grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 16 messages


science_image = [2020021900271]


In [138]:
#Rotation on Firefly display seems to work fine. 
await attcs.offset_azel(az = 0, el = 30) #units are arcseconds 

In [139]:
#And slew to object with rot_pa = 0.0  
science_image = await latiss.take_object(exptime=10., n=1, filter='empty_1', grating='ronchi90lpmm')
print ('science_image = ' + str(science_image))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


science_image = [2020021900272]


In [140]:
#Now try to slew again to the target.  Tracking should automatically resume. 
target_name = "HD 45348"
await attcs.slew_object(name = target_name,  rot_sky=0.0, slew_timeout=240.)

RemoteEvent(ATMCS, 0, target) falling behind; read 12 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 12 messages
atptg not in <State.ENABLED: 2>: <State.FAULT: 3>
RuntimeError, cancel_not_done.


RuntimeError: atptg state is <State.FAULT: 3>, expected <State.ENABLED: 2>

Task exception was never retrieved
future: <Task finished coro=<ATTCS.monitor_position() done, defined at /home/saluser/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py:1022> exception=TimeoutError()>
Traceback (most recent call last):
  File "/home/saluser/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py", line 1049, in monitor_position
    in_position = False
  File "/home/saluser/repos/ts_salobj/python/lsst/ts/salobj/topics/read_topic.py", line 384, in next
    return await self._next(timeout=timeout)
  File "/home/saluser/repos/ts_salobj/python/lsst/ts/salobj/topics/read_topic.py", line 397, in _next
    await asyncio.wait_for(self._next_task, timeout=timeout)
  File "/opt/lsst/software/stack/python/miniconda3-4.7.10/envs/lsst-scipipe-4d7b902/lib/python3.7/asyncio/tasks.py", line 423, in wait_for
    raise futures.TimeoutError()
concurrent.futures._base.TimeoutError


In [141]:
await salobj.set_summary_state(attcs.atptg, salobj.State.ENABLED)  

RuntimeError: Error on cmd=cmd_enable, initial_state=3: msg='Command failed', ackcmd=(ackcmd private_seqNum=471230478, ack=<SalRetCode.CMD_FAILED: -302>, error=6612, result='Rejected : command not allowed in current state')

In [142]:
#Now try to slew again to the target.  Tracking should automatically resume. 
target_name = "HD 45348"
await attcs.slew_object(name = target_name,  rot_sky=0.0, slew_timeout=240.)

In [144]:
#Go way out of focus to get a huge donut 
focus_pos = 3.9
attcs.athexapod.evt_positionUpdate.flush() 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
focus_pos = 4.0
attcs.athexapod.evt_positionUpdate.flush() 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)

In [146]:
science_image = await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


In [147]:
print ('science_image = ' + str(science_image))

science_image = [2020021900273]


In [148]:
#Taking 10
science_images = await latiss.take_object(exptime=0.25, n=10, filter='BG40',grating='empty_1')
print ('science_images = ' + str(science_images))

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


science_images = [2020021900274 2020021900275 2020021900276 2020021900277 2020021900278
 2020021900279 2020021900280 2020021900281 2020021900282 2020021900283]


In [149]:
focus_pos = 1.5
attcs.athexapod.evt_positionUpdate.flush() 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)

In [150]:
science_image = await latiss.take_object(exptime=0.25, n=1, filter='empty_1',grating='empty_1')

In [151]:
focus_pos = 1.
attcs.athexapod.evt_positionUpdate.flush() 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)

In [152]:
science_image = await latiss.take_object(exptime=0.25, n=1, filter='empty_1',grating='empty_1')

In [154]:
science_image = await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


In [155]:
print ('science_image = ' + str(science_image))

science_image = [2020021900286]


In [156]:
await attcs.offset_xy(x = -20, y = 0)

In [157]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

array([2020021900287])

In [158]:
await attcs.offset_xy(x = -13, y = 0)

In [159]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

array([2020021900288])

In [160]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 + 50, y = 0)

In [161]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1') 

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


array([2020021900289])

In [162]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 + 50 + 50, y = 0)

In [163]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1') 

array([2020021900290])

In [164]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 + 50 + 50 + 50, y = 0)

In [165]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1') 

array([2020021900291])

In [166]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50, y = 0)

In [167]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1') 

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages


array([2020021900292])

In [168]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50, y = 0)

In [169]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1') 

array([2020021900293])

In [170]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50 - 48, y = 0)

In [171]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1') 

array([2020021900294])

In [172]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50 - 48 - 48 , y = 0)

In [173]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

array([2020021900295])

In [174]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50 - 48 - 48 , y = -150)

In [175]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


array([2020021900296])

In [176]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50 - 48 + 45 , y = -150)

In [177]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

array([2020021900297])

In [178]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50 - 48 - 48 + 45 , y = -150)

In [179]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

array([2020021900298])

In [180]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50 - 48 - 48 + 45 + 90, y = -150)

In [181]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


array([2020021900299])

In [182]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50 - 48 - 48 + 45 + 90 + 10, y = -150)

In [183]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

array([2020021900300])

In [184]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50 - 48 - 48 + 45 + 90 + 10 + 50, y = -150)

In [185]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


array([2020021900301])

In [186]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50 - 48 - 48 + 45 + 90 + 10 + 50 + 45, y = -150)

In [187]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


array([2020021900302])

In [188]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50 - 48 - 48 + 45 + 90 + 10 + 50 + 45 + 45, y = -150)

In [189]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

array([2020021900303])

In [190]:
# x = -13, y = 0 is our starting offset
await attcs.offset_xy(x = -13 -50 - 50 - 48 - 48 + 45 + 90 + 10 + 50 + 45 + 45 + 45, y = -150)

In [191]:
await latiss.take_object(exptime=0.25, n=1, filter='BG40',grating='empty_1')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


array([2020021900304])

In [192]:
focus_pos = -0.1155
attcs.athexapod.evt_positionUpdate.flush() 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)

In [193]:
focus_pos = -0.1155
attcs.athexapod.evt_positionUpdate.flush() 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)

RemoteTelemetry(ATDome, 0, position) falling behind; read 16 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


In [ ]:
#Tiago took over for a time. 

In [194]:
#Going to the zenith 
target_name = "HD 116351"
await attcs.slew_object(name = target_name,  rot_sky=0.0, slew_timeout=240.)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1563232240, ack=<SalRetCode.CMD_FAILED: -302>, error=6612, result='Rejected : command not allowed in current state')

In [195]:
await salobj.set_summary_state(attcs.atptg, salobj.State.ENABLED) 

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [196]:
await attcs.atptg.cmd_focusName.set_start(focus=3) #focus = 3 selects Nasmyth port of pointing 

In [197]:
#Going to the zenith 
target_name = "HD 116351"
await attcs.slew_object(name = target_name,  rot_sky=0.0, slew_timeout=240.)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1563232241, ack=<SalRetCode.CMD_FAILED: -302>, error=6611, result='Rejected : rotator position angle out of range')

In [198]:
#Going to the zenith 
target_name = "HD 116351"
await attcs.slew_object(name = target_name,  rot_sky=-180.0, slew_timeout=240.)

In [200]:
await latiss.take_object(exptime=20, n=1, filter='BG40',grating='ronchi170lpmm')

array([2020021900311])

In [ ]:
#Robert calculates offset 

In [201]:
await attcs.offset_xy(x=-39, y = -25)

In [202]:
await latiss.take_object(exptime=20, n=1, filter='BG40',grating='ronchi170lpmm')

array([2020021900312])

In [204]:
await latiss.take_object(exptime=20, n=1, filter='quadnotch1',grating='ronchi170lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


array([2020021900313])

In [205]:
#Now move down to get the begining of the quad not spectrum in position
await attcs.offset_xy(x=-39, y = -25 - 75)

In [206]:
await latiss.take_object(exptime=20, n=1, filter='quadnotch1',grating='ronchi170lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


array([2020021900314])

In [207]:
await latiss.take_object(exptime=120, n=1, filter='quadnotch1',grating='ronchi170lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


array([2020021900315])

In [208]:
await latiss.take_object(exptime=120, n=1, filter='BG40',grating='ronchi170lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


array([2020021900316])

In [212]:
#Now move down to get the begining of the quad not spectrum in position
await attcs.offset_xy(x=-39, y = -25 - 75 - 90)

In [213]:
await latiss.take_object(exptime=120, n=1, filter='RG610',grating='ronchi170lpmm')

array([2020021900317])

In [214]:
#Now move down to get the begining of the quad not spectrum in position
await attcs.offset_xy(x=-39, y = -25 - 75 - 90 + 20)

In [215]:
await latiss.take_object(exptime=120, n=1, filter='RG610',grating='ronchi170lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


array([2020021900318])

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages


In [216]:
await latiss.take_object(exptime=50, n=12, filter='RG610',grating='empty_1')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


array([2020021900319, 2020021900320, 2020021900321, 2020021900322,
       2020021900323, 2020021900324, 2020021900325, 2020021900326,
       2020021900327, 2020021900328, 2020021900329, 2020021900330])

In [ ]:
#EXECUTE THE FOLLOWNG WHEN YOU'VE GOT AN OBJECT IN POSITION ON WHICH YOU WANT TO FOCUS 

In [ ]:
#Focus units are given in mm 
focus_center = -0.1155
focus_step = 0.02
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * math.floor(n_steps / 2), focus_center + focus_step * math.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str(focus_positions))

In [ ]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    focus_image = await latiss.take_engtest(exptime=15., n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(focus_image) )
    focus_images[i] = int(focus_image)
    
#This resets the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

In [ ]:
#Measure centroids 
#If you get some version of the error: 
#     ModuleNotFoundError: No module named 'lsst.atmospec'
# that probably means you are working off of the Python3 kernel, rather than the local kernel.  
#We found this would happen when the kernel crashed and had to be restarted.
#This problem can be easily fixed by clicking on the "Python 3" in the upper right of your jupyter notebook window.  

from lsst.atmospec.processStar import ProcessStarTask
config = ProcessStarTask.ConfigClass()
config.mainSourceFindingMethod = 'ROUNDEST'
task = ProcessStarTask(config=config)

In [ ]:
all_sub_arrays = [[] for img in focus_images]
image_arrays = [[] for img in focus_images] 
centroids = [[] for img in focus_images]
obj_name = '' #We assume that we are focusing on one and only one object.  Pulled from first image in sequence 
for i in range(len(focus_images)):
    img = focus_images[i] 
    print ('Reading in spectrum from science_image ' + str(img))
    seq_str = int(str(img)[9:]) #Truncate the leading date string, and lose extra 0s
    dataId = dict(dayObs=date_str)
    dataId["seqNum"] = seq_str
    if apply_bias: 
        try:
            bias
        except NameError:
            bias = butler.get('bias', dataId)
        exp = butler.get('raw', dataId)
        exp = isrTask.run(exp, bias = bias).exposure
    else: 
        try:
            bias
        except NameError:
            bias = butler.get('bias', dataId)
        exp = butler.get('raw', dataId)
        exp = isrTask.run(exp).exposure
    obj_name = exp.getMetadata().toDict()["OBJECT"]
    array = np.transpose(exp.image.array) 
    image_arrays[i] = array
    centroid = task.findMainSource(exp)
    centroids[i] = [int(centroid[0]), int(centroid[1])] 
    centroid = centroids[i] 
    sub_arrays = [array[centroid[0] + spectrum_fit_col_edges[0]:centroid[0] + spectrum_fit_col_edges[1], centroid[1] + spectrum_offsets[j]:centroid[1] + spectrum_offsets[j] + binning] for j in range(len(spectrum_offsets))]
    all_sub_arrays[i] = sub_arrays
    
print ('centroids = ' + str(centroids))

In [ ]:
#Do the fits to the seeing 
fits = [[[] for spectrum_offset in spectrum_offsets] for img in focus_images]

f, axarr = plt.subplots(len(all_sub_arrays), len(spectrum_offsets), figsize = [10,15])
for i in range(len(all_sub_arrays)):
    centroid = centroids[i] 
    sub_arrays = all_sub_arrays[i]
    print ('Fitting sub array  ' + str(i)) 
    for j in range(len(spectrum_offsets)): 
        sub_array = sub_arrays[j]
        row = np.sum(sub_array, axis = 1) 
        xs = range(len(row))
        max_col = np.argmax(row)
        sorted_row = sorted(row)
        floor_estimate = np.mean(sorted_row[0:int(len(sorted_row) * lower_fraction_to_measure_floor)])
        bias_sub_row = np.array(row) - floor_estimate 
        init_guess = [row[max_col], max_col, arcmin_to_pixel * seeing_guess_in_arcmin / np.log(np.sqrt(2.0))]
        try:
            best_fit =  optimize.curve_fit(lambda xs, A, mu, sig: fitting_funct(xs, A, mu, sig, floor_estimate), xs, bias_sub_row, p0=init_guess, maxfev = 10000) 
            fit_params = list(best_fit[0]) + [0] 
        except RuntimeError:
            print ('Failed to fit bin ' + str(bin) + '.  Returning initial best guess fit.')
            fit_params = init_guess + [1]
        fits[i][j] = fit_params
        axarr[i,j].scatter(xs, bias_sub_row)
        axarr[i,j].plot(xs, fitting_funct(xs, *fit_params[0:-1], floor_estimate))
        axarr[i,j].set_xlabel('column (pix)')
        axarr[i,j].set_ylabel('Binned ADU w/ nBin = ' + str(binning))
        
plt.show( )
print ('fits = ' + str(fits))

In [ ]:
#Now we want to make plots of fit vs focus
#We will plot this
sigma_to_fwhm = 2.35
sec_heights = [[0.0 for pos in focus_positions] for spec_sec in spectrum_offsets]
sec_widths = [[0.0 for pos in focus_positions] for spec_sec in spectrum_offsets]
sec_centers = [[0.0 for pos in focus_positions] for spec_sec in spectrum_offsets]
for fit_num in range(len(fits)):
    single_obj_fit = fits[fit_num]
    for offset_num in range(len(spectrum_offsets)): 
        fit = single_obj_fit[offset_num]
        sec_heights[offset_num][fit_num] = fit[0] 
        sec_widths[offset_num][fit_num] = fit[2] 
        sec_centers[offset_num][fit_num] = fit[1] 
#print ('sec_heights = ' + str(sec_heights ))
#print ('sec_widths = ' + str(sec_widths ))
#print ('sec_centers = ' + str(sec_centers ))

#Now we want to fit the widths of each spectrum section 
quadratic_fits_of_widths = [[] for offset in spectrum_offsets]
for offset_num in range(len(spectrum_offsets)):
    single_sec_widths = sec_widths[offset_num]
    quadratic_fit = np.polyfit(focus_positions, single_sec_widths, 2)
    quadratic_fits_of_widths[offset_num] = quadratic_fit
    fit_min = -quadratic_fit[1] / (2.0 * quadratic_fit[0])
    print ('For ' + str(offset_num) + 'th offset, fitted minimium = ' + str(round_to_n(fit_min, 3)) + 'mm')
    
#Now plot the fits and the seeing data points 
f, axarr = plt.subplots(n_free_fit_params-1,1, figsize = [10, 5])
colors = ['b', 'orange', 'r']
xlabels = ['Focus position (mm)', 'Focus position (mm)']
ylabels = ['Height (binned ADU)', 'FWHM (arcsec)']
param_strs = ['Height (binned ADU)', 'Center (col)', 'Sigma width (pix)']
param_legends = ['m=+1 blue end', 'm=+1 middle', 'm=+1 red end']
scats_for_legend = [0.0 for offset in spectrum_offsets]
for offset_num in range(len(spectrum_offsets)): 
    scats_for_legend[offset_num] = axarr[0].scatter(focus_positions, sec_heights[offset_num], c = colors[offset_num])
    axarr[0].set_xlabel(xlabels[0], fontsize = title_font_size)
    axarr[0].set_ylabel(ylabels[0], fontsize = label_font_size)
    axarr[0]
    #axarr[1].scatter(focus_positions, sec_centers[offset_num], c = colors[offset_num])
    axarr[1].scatter(focus_positions, [center * 1.0 / arcmin_to_pixel * sigma_to_fwhm for center in sec_widths[offset_num] ] , c = colors[offset_num])
    axarr[1].plot(focus_positions, np.poly1d(quadratic_fits_of_widths[offset_num])(focus_positions) * 1.0 / arcmin_to_pixel * sigma_to_fwhm, c = colors[offset_num])
    axarr[1].set_xlabel(xlabels[1])
    axarr[1].set_ylabel(ylabels[1]) 
plt.suptitle('Focus curve for spectrum of ' + obj_name, fontsize = title_font_size)
axarr[0].legend(scats_for_legend, param_legends )
axarr[1].legend(scats_for_legend, param_legends )
f.tight_layout(rect=[0, 0.03, 1, 0.95] )
plt.show() 

In [217]:
await attcs.shutdown() 

CancelledError: 

RemoteEvent(ATHexapod, 0, summaryState) falling behind; read 79 messages
RemoteEvent(ATHexapod, 0, settingVersions) falling behind; read 65 messages
